In [ ]:
#Dùng để get dữ liệu từ Facebook API
import facebook
import requests

#Dùng để get thời gian
from datetime import datetime, timedelta
import time

#Dùng để work với gg sheet
import pygsheets

#Một số thư viện khác
import pandas as pd

In [ ]:
#Mở gg sheet
client = pygsheets.authorize(service_file = '/Users/genkin/Downloads/MAIDO BOT/maido-agency-e7cc6f08cb95.json')
sh = client.open('API BOT Tracking')

# sh = client.open('MAIDO - DB MAGIC DATA')
wks = sh.worksheet_by_title('FB Accounts')

#Lấy dữ liệu từ CampaignPeriod và lưu thành DataFrame
df = wks.get_as_df()

#Loại bỏ những dòng có access_token là null
drop_rows = df[df['access_token']==''].index
df.drop(drop_rows,axis = 0,inplace = True)

#Loại bỏ các account khác
rows = df[df['name'] != 'HT'].index
df.drop(rows,axis = 0,inplace = True)

In [ ]:
def get_page_access_token(access_token):
  token_lst = []
  id_lst =[]
  name_lst = []
  query = "https://graph.facebook.com/v15.0/me?fields=accounts.limit(1000){access_token,id,name}&access_token={access_token}"
  run_query = query.replace("{access_token}",access_token)

  r = requests.get(run_query).json()
  for i in r['accounts']['data']:
    token_lst.append(i['access_token'])
    id_lst.append(i['id'])
    name_lst.append(i['name'])

  return token_lst, id_lst, name_lst

In [ ]:
#Lấy ra new_conversation_unique, date với params truyền vào là ngày bắt đầu, mặc định là start:2022-10-01 và end:today
def get_new_conversation_unique(access_token, start, end):
    query = f'me/insights/page_messages_new_conversations_unique/?since={start}&until={end}'

    #Gọi graph API và requests
    graph = facebook.GraphAPI(access_token)    
    new_conversation_unique_query = graph.request(query)

    #Tạo list rỗng để lưu các giá trị 
    date = []
    value = []

    #Lặp qua tất cả giá trị trả về, lưu ý chỉ lấy phần tử đầu tiên (index =0) vì kế quả trả về là list gồm 3 phần tử, tạm gọi là phân theo daily(quan tâm), weekly(ko quan tâm), monthly(ko quan tâm)
    for i in new_conversation_unique_query['data'][0]['values']:

      #date: Loại bỏ dấu giờ gian bắt đầu bằng chữ T**** trước khi thêm vào list
      timestamp = i['end_time']
      formated_date = timestamp[0:timestamp.find('T')]
      date.append(formated_date)

      #page_messages_new_conversations_unique
      value.append(i['value'])
    
    return date, value

#Lấy ra page_like với params truyền vào là ngày bắt đầu, số ngày tối đa cho phép là 90 ngày (3 tháng) nên sẽ tách ra thành 2 query
def get_page_like_unique(access_token, start, end):
    query = f'me/insights/page_fan_adds_unique/?since={start}&until={end}'

    #Gọi graph API và requests
    graph = facebook.GraphAPI(access_token)    
    page_like_unique_query = graph.request(query)
    
    #Tạo list rỗng để lưu các giá trị 
    value = []

    #Lặp qua tất cả giá trị trả về, lưu ý chỉ lấy phần tử đầu tiên (index =0) vì kế quả trả về là list gồm 3 phần tử, tạm gọi là phân theo daily(quan tâm), weekly(ko quan tâm), monthly(ko quan tâm)
    for i in page_like_unique_query['data'][0]['values']:
      value.append(i['value'])
    
    return value

#Lặp qua cả DataFrame để requests các giá trị từ access_token
def fb_page(df,start, end):
  #Tạo một DataFrame rỗng
  data = pd.DataFrame(columns = ["date","page_name","new_conversation_unique","page_like_unique"])

  for i in range(0,len(df)):
    access_token = df.iloc[i]['access_token']

    token_lst, id_lst, name_lst = get_page_access_token(access_token)

    for token,id,name in zip(token_lst,id_lst,name_lst):
      print('Đang lấy data từ Page:', name)
      #Lấy ra các insights
      date, new_conversation_unique = get_new_conversation_unique(token, start, end)
      page_like_unique = get_page_like_unique(token, start, end)

      #page_name phải * với len(...) vì để list page_name có độ dài bằng với độ dài các list khác => Gộp thành DataFrame
      page_name = [name]*len(date)

      #Tạo một DataFrame tạm để append vào DataFrame chính (data)
      temp_data = pd.DataFrame({'date':date, 'page_name':page_name,'new_conversation_unique':new_conversation_unique, 'page_like_unique':page_like_unique}, columns = ["date","page_name","new_conversation_unique","page_like_unique"])
      data = pd.concat([data,temp_data], axis =0)

  print('--------------------')
  return data

In [ ]:
while True:
  output = pd.DataFrame(columns = ['date','page_name','new_conversation_unique','page_like_unique'])
  today =  datetime.strptime(datetime.today().strftime('%Y-%m-%d'),'%Y-%m-%d')
  date_lst = ['2022-09-30']
  while datetime.strptime(date_lst[-1], '%Y-%m-%d') < today:
    date_lst.append((datetime.strptime(date_lst[-1], '%Y-%m-%d') + timedelta(90)).strftime('%Y-%m-%d'))

    if datetime.strptime(date_lst[-1], '%Y-%m-%d') >= today:
      date_lst[-1] = today.strftime('%Y-%m-%d')
      break



  for i in range(0,len(date_lst)-1):
    print('Lấy data ngày:', datetime.strptime(date_lst[i], '%Y-%m-%d') + timedelta(1),"-",datetime.strptime(date_lst[i+1], '%Y-%m-%d'))
    data = fb_page(df,date_lst[i],date_lst[i+1])
    output = pd.concat([output,data], axis =0)

  tracking_sh = client.open('API BOT Tracking')

  write_sheet = sh.worksheet_by_title('FB Page BOT')
  write_sheet.set_dataframe(output,start = "A1",copy_head =True)

  print('------DONE------')
  print("Nghỉ 1 tiếng rồi chạy tiếp")
  print('')
  time.sleep(3600)

Lấy data ngày: 2022-10-01 00:00:00 - 2022-12-29 00:00:00
Đang lấy data từ Page: Hanah Beauty Clinics 
Đang lấy data từ Page: Fitmates Times City - Gym Studio
Đang lấy data từ Page: Xuka Sashimi
Đang lấy data từ Page: Kim Long
Đang lấy data từ Page: AN Grill and Hotpot - Lẩu Nướng chuẩn vị Nhật Bản
Đang lấy data từ Page: VIGO5 Fitness & Yoga Tân Tây Đô
Đang lấy data từ Page: Thế giới Arnica
Đang lấy data từ Page: Thiệp Cưới Biên Hoà - thiepcuoibienhoa.com
Đang lấy data từ Page: Thỏ Xinh Clothing
Đang lấy data từ Page: Ginza Garden - Khu Ẩm Thực Nghỉ Dưỡng
Đang lấy data từ Page: Trần Công Đệ 
Đang lấy data từ Page: Thiền Vipassana - Thiền Tứ Niệm Xứ
Đang lấy data từ Page: Hàng Thiết Kế Store
Đang lấy data từ Page: Đèn Á Âu
Đang lấy data từ Page: Dược liệu Hải Hậu ACT
Đang lấy data từ Page: Shop Chuyên Tinh Dầu Nước Hoa Pháp
Đang lấy data từ Page: CÀ PHÊ SẠCH FARMER COFFEE
Đang lấy data từ Page: The Ruộng Resort
Đang lấy data từ Page: Littleink
Đang lấy data từ Page: Ba Cô Store - Cửa hàn